In [1]:
from peft import PeftConfig
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn as nn

2025-03-01 18:37:34.011629: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 18:37:34.026699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740843454.043170  600690 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740843454.048084  600690 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 18:37:34.065855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
from peft import PeftModel

from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn as nn

from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM
import torch.nn as nn

class MyAdapterConfig(PeftConfig):
    def __init__(self, custom_param=None, **kwargs):
        super().__init__(**kwargs)
        self.peft_type = "MY_ADAPTER"
        self.custom_param = custom_param

    @staticmethod
    def from_dict(obj):
        return MyAdapterConfig(**obj)

    def to_dict(self):
        return {
            **super().to_dict(),
            "custom_param": self.custom_param,
        }

class MyAdapterModel(PeftModel):
    def __init__(self, model, peft_config):
        super(PeftModel, self).__init__()
        self.base_model = model
        self.peft_config = peft_config
        self.config = model.config
        self._is_trainable = True  # Установите этот флаг при необходимости

        # Ваша модификация слоев модели
        transformer_layers = self.base_model.transformer.h

        for idx, layer in enumerate(transformer_layers):
            modified_layer = nn.Sequential(
                layer,
                nn.Linear(self.base_model.config.hidden_size, self.base_model.config.hidden_size)
            )
            transformer_layers[idx] = modified_layer
            
    def save_pretrained(self, save_directory):
    # Сохраните конфигурацию адаптера
        self.peft_config.save_pretrained(save_directory)
    # Сохраните веса модели
        self.base_model.save_pretrained(save_directory)
        
    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs):
    # Загрузите конфигурацию адаптера
        peft_config = MyAdapterConfig.from_pretrained(pretrained_model_name_or_path)
    # Загрузите базовую модель
        base_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)
    # Создайте экземпляр вашей модели
        model = cls(base_model, peft_config)
        return model

    def forward(self, *args, **kwargs):
        return self.base_model(*args, **kwargs)

In [3]:
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Создайте конфигурацию адаптера
peft_config = MyAdapterConfig(custom_param="value")

# Оберните базовую модель вашим адаптером
model = MyAdapterModel(base_model, peft_config)

AttributeError: 'GPT2LMHeadModel' object has no attribute '_is_prompt_learning'

In [ ]:
print(model)

In [ ]:
model.save_pretrained("path_to_save")

In [ ]:
from peft import PeftConfig

# Загрузите конфигурацию адаптера
peft_config = PeftConfig.from_pretrained("path_to_save")

# Загрузите базовую модель
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Проверьте тип адаптера и загрузите модель соответствующим образом
if peft_config.peft_type == "MY_ADAPTER":
    model = MyAdapterModel(base_model, peft_config)
else:
    model = PeftModel.from_pretrained(base_model, "path_to_save")

In [3]:
import torch

def PISSA(W, r):
    # Проверка r на допустимость
    min_dim = min(W.shape)
    if r > min_dim:
        raise ValueError(f"Ранг r не может быть больше минимального измерения матрицы ({min_dim}).")

    # Вычисляем сингулярное разложение матрицы W
    U, S, Vh = torch.linalg.svd(W, full_matrices=False)
    
    # Обрезаем сингулярные значения и векторы до ранга r
    U_r = U[:, :r]
    S_r = S[:r]
    Vh_r = Vh[:r, :]
    
    # Раскладываем сингулярные значения по U и V с использованием квадратного корня
    S_root = torch.sqrt(S_r)
    
    # Умножение с правильными размерностями
    U_mod = U_r @ torch.diag_embed(S_root)
    V_mod = torch.diag_embed(S_root) @ Vh_r

    return U_mod, V_mod

# Пример вызова функции
W = torch.randn(8, 4)  # Пример 8x4 матрицы
r = 2  # Указываем ранг
U_mod, V_mod = PISSA(W, r)

print("Shapes:")
print("U_mod:", U_mod.shape)
print("V_mod:", V_mod.shape)

Shapes:
U_mod: torch.Size([8, 2])
V_mod: torch.Size([2, 4])


In [18]:
import torch
import torch.nn as nn

X = torch.randn(2, 3, 4)

W = torch.randn(5, 4)

linear_layer = nn.Linear(4, 5)

with torch.no_grad():
    linear_layer.weight.copy_(W)
    linear_layer.bias.zero_()       # Обнуляем смещение

print(X @ W.T)
print()
print()
print(linear_layer(X))

# X = X.permute(2, 1, 0).reshape(4, 2 * 3)
# W = W.T

# print(X.shape, W.shape)

# print(W @ X)


tensor([[[-1.5443, -1.3908,  0.6518, -3.3188,  0.4085],
         [ 1.6499,  0.0765, -2.7106, -3.1296,  0.3357],
         [ 0.8834,  3.0060,  1.7151,  1.6291,  0.1424]],

        [[ 0.2220,  1.8592,  0.9303,  2.6423, -0.2677],
         [ 0.0930,  4.7357,  0.6181,  0.6838, -0.0851],
         [ 1.9090, -0.7330, -1.9167,  1.6740, -0.2512]]])


tensor([[[-1.5443, -1.3908,  0.6518, -3.3188,  0.4085],
         [ 1.6499,  0.0765, -2.7106, -3.1296,  0.3357],
         [ 0.8834,  3.0060,  1.7151,  1.6291,  0.1424]],

        [[ 0.2220,  1.8592,  0.9303,  2.6423, -0.2677],
         [ 0.0930,  4.7357,  0.6181,  0.6838, -0.0851],
         [ 1.9090, -0.7330, -1.9167,  1.6740, -0.2512]]],
       grad_fn=<ViewBackward0>)
